In [ ]:
from datetime import datetime
import pandas as pd
import os

In [ ]:
def set_google_credentials(CONFIG, GOOGLE_CREDENTIALS):
    try:
        os.environ['GOOGLE_APPLICATION_CREDENTIALS']
    except:
        print(f"setting google credentials as global variable...")
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = CONFIG \
        + GOOGLE_CREDENTIALS or input("No Google API credendials file provided." 
        + "Please specify path now:\n")
        
set_google_credentials("./config/","google-creds.json")

In [ ]:
query = """
    SELECT
      DISTINCT * FROM (
        SELECT
          DISTINCT *
        FROM
          `ethereum-data-nero.ethdata.beaconchain_pace`
        WHERE
          slot IN (
          SELECT
            slot
          FROM
            `ethereum-data-nero.eth.validator_info`
          WHERE
            validator_id= "missed"
            )
        and validator_id != 0
    )
    ORDER BY slot
"""
df = pd.read_gbq(query)
def slot_in_epoch(slot):
    return slot%32
df["slot_in_epoch"] = df["slot"].apply(slot_in_epoch)
df['epoch'] = df['slot'] // 32

In [ ]:
len(df), min(df.slot), max(df.slot)

In [ ]:
all_epochs = list(range(min(df["epoch"]), max(df["epoch"])+1))

# Probability that slot 2 is reorged
P_R2 = len(df[df.slot_in_epoch == 1]) / len(all_epochs)
P_R3 = len(df[df.slot_in_epoch == 2]) / len(all_epochs)
P_R4 = len(df[df.slot_in_epoch == 3]) / len(all_epochs)

# Identify epochs where slot 2 was reorged
epochs_with_R2 = set(df[df.slot_in_epoch == 1].epoch.tolist())
epochs_with_R3 = set(df[df.slot_in_epoch == 2].epoch.tolist())
epochs_with_R4 = set(df[df.slot_in_epoch == 3].epoch.tolist())
epochs_with_R2_R3 = epochs_with_R2 & epochs_with_R3

# Probability that slot 3 is reorged given slot 2 was reorged
P_R3_given_R2 = len(df[(df.slot_in_epoch == 2) & df.epoch.isin(epochs_with_R2)]) / len(epochs_with_R2)

epochs_without_R2 = list(set(all_epochs) - set(epochs_with_R2))

# Probability that slot 3 is reorged given slot 2 was not reorged
P_R3_given_not_R2 = len(df[(df.slot_in_epoch == 2) & df.epoch.isin(epochs_without_R2)]) / len(epochs_without_R2)


# Probability that slot 4 is reorged given slot 2 was reorged
P_R4_given_R2 = len(df[(df.slot_in_epoch == 3) & df.epoch.isin(epochs_with_R2)]) / len(epochs_with_R2)

# Probability that slot 4 is reorged given slot 3 was reorged
P_R4_given_R3 = len(df[(df.slot_in_epoch == 3) & df.epoch.isin(epochs_with_R3)]) / len(epochs_with_R3)

# Probability that slot 4 is reorged given neither slot 2 nor slot 3 was reorged
epochs_without_R2_R3 = list(set(all_epochs) - set(epochs_with_R2) - set(epochs_with_R3))
P_R4_given_not_R2_R3 = len(df[(df.slot_in_epoch == 3) & df.epoch.isin(epochs_without_R2_R3)]) / len(epochs_without_R2_R3)

epochs_without_R2 = list(set(all_epochs) - set(epochs_with_R2))
P_R4_given_not_R2 = len(df[(df.slot_in_epoch == 3) & df.epoch.isin(epochs_without_R2)]) / len(epochs_without_R2)

epochs_without_R3 = list(set(all_epochs) - set(epochs_with_R3))
P_R4_given_not_R3 = len(df[(df.slot_in_epoch == 3) & df.epoch.isin(epochs_without_R3)]) / len(epochs_without_R3)

P_R4_given_R2_R3 = len(df[(df.slot_in_epoch == 3) & df.epoch.isin(epochs_with_R2_R3)]) / len(epochs_with_R2_R3)

print("P(R2):", P_R2)
print("P(R3):", P_R3)
print("P(R4):", P_R4)
print("P(R3 | R2):", P_R3_given_R2)
print("P(R3 | ¬R2):", P_R3_given_not_R2)
print("P(R4 | R2):", P_R4_given_R2)
print("P(R4 | R3):", P_R4_given_R3)
print("P(R4 | R2 & R3):", P_R4_given_R2_R3)
print("P(R4 | ¬R2 & ¬R3):", P_R4_given_not_R2_R3)
print("RR_R2_R3", P_R3_given_R2/P_R3_given_not_R2)
print("RR_R2_R4", P_R4_given_R2/P_R4_given_not_R2)
print("RR_R3_R4", P_R4_given_R3/P_R4_given_not_R3)
